# Cental Bank Exchange Rates

## Daily scraping

In [1]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
webpage_url = 'https://www.cbsl.gov.lk/en/rates-and-indicators/exchange-rates'

# Send a request to fetch the webpage
response = requests.get(webpage_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the link using the provided CSS selector
css_selector = '#article-146 > div > div > div > div > div:nth-child(5) > div:nth-child(2) > p:nth-child(2) > a'
link_element = soup.select_one(css_selector)

if link_element:
    # Extract the href attribute (the URL of the Excel sheet)
    excel_url = link_element.get('href')
    
    # Handle relative URLs (if necessary)
    if not excel_url.startswith('http'):
        excel_url = requests.compat.urljoin(webpage_url, excel_url)
    
    # Download the Excel sheet
    excel_response = requests.get(excel_url)
    
    # Save the Excel sheet to a local file
    with open('central_bank_exchangerates_file.xlsx', 'wb') as file:
        file.write(excel_response.content)
    
    print(f"Excel sheet downloaded successfully: {excel_url}")
else:
    print("Link not found.")


Excel sheet downloaded successfully: https://www.cbsl.gov.lk/sites/default/files/cbslweb_documents/statistics/sheets/Buying_&_Selling_Exchange_Rates_20240802.xlsx


## Download excel content

In [2]:
from io import BytesIO

response = requests.get(excel_url)
response.raise_for_status() 

excel_content=BytesIO(response.content)

## Excel to Dataframe

In [3]:
import pandas as pd

df1=pd.read_excel(excel_content)


In [4]:
# Drop columns named 'Unnamed: 0'
df1= df1.loc[:, df1.columns != 'Unnamed: 0']

In [5]:
df1 = df1.drop(index=[0, 1, 2])

In [6]:
# Fill NaN values with empty strings for both rows
first_row_filled = df1.iloc[0].fillna('')
second_row_filled = df1.iloc[1].fillna('')

# Combine the values from the first and second rows for each column, ignoring NaNs
combined_row = first_row_filled + '_' + second_row_filled

# Update the second row with the combined values
df1.iloc[1] = combined_row

df1.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
3,NaN,Buying,Selling,Buying,Selling,Buying,Selling,Buying,Selling,Buying,Selling,Buying,Selling,Buying,Selling,Buying,Selling,Buying,Selling
4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN


In [7]:
df1 = df1.drop(index=3)
df1.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
8,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN


In [8]:
# Drop rows where all cells are NaN
df1 = df1.dropna(how='all')

df1.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
8,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
9,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN


In [9]:
df1.columns = df1.iloc[0] 
df1.head()
# Set the first row as the header
# df1 = df1.drop(index=4).reset_index(drop=True)  # Drop the first row and reset the index
# df1.head()

4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
8,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
9,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN


In [10]:
df1.reset_index()

4,index,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
0,4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
1,6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
2,7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
3,8,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
4,9,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,5192,2024-07-30 00:00:00,298.0989,307.3942,381.6134,396.6713,320.6294,334.3416,333.2026,349.6845,213.7031,223.3603,193.2052,203.174,219.944,230.4794,1.9248,2.0057,40.2655,42.8347
4954,5193,2024-07-31 00:00:00,297.8582,307.1744,381.1929,396.2421,320.7613,334.2902,334.8863,351.0156,213.7335,223.2639,191.8704,201.6048,220.2266,230.6142,1.9419,2.0206,40.3414,43.0128
4955,5194,Average July,299.067722,308.3477,382.900483,397.847817,322.429661,336.044513,332.180248,348.181765,216.690217,226.326309,197.635852,207.750574,220.216683,230.671548,1.883635,1.962009,40.32543,42.980596
4956,5196,(a) The average buying and selling exchange ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df1 = df1.dropna(axis=1, how='all')
df1.head()

4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
8,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
9,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN


In [12]:
# Clean up column names: remove extra spaces after underscores
df1.columns = [col.replace('_', '_').replace(' ', '') for col in df1.columns]

In [13]:
df1.head()

,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_EUR,Buying_CHF,Selling_CHF,Buying_CAD,Selling_CAD,Buying_AUD,Selling_AUD,Buying_SGD,Selling_SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
4,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_ EUR,Buying_ CHF,Selling_ CHF,Buying_ CAD,Selling_ CAD,Buying_AUD,Selling_AUD,Buying_ SGD,Selling_ SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
6,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
7,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
8,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
9,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN


In [14]:
df1 = df1.drop(index=4).reset_index(drop=True)  # Drop the first row and reset the index
df1.head()

,_,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_EUR,Buying_CHF,Selling_CHF,Buying_CAD,Selling_CAD,Buying_AUD,Selling_AUD,Buying_SGD,Selling_SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
0,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
1,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
2,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
3,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN
4,2005-01-07 00:00:00,98.58,100.24,184.49,188.64,129.38,132.71,83.34,86.06,NaN,NaN,74.59,76.93,NaN,NaN,0.9358,0.96117,NaN,NaN


In [15]:
# Rename columns named '-' to 'Date'
df1_cleaned = df1.rename(columns={'_': 'Date'})

df1_cleaned.head(40)


,Date,Buying_USD,Selling_USD,Buying_GBP,Selling_GBP,Buying_EUR,Selling_EUR,Buying_CHF,Selling_CHF,Buying_CAD,Selling_CAD,Buying_AUD,Selling_AUD,Buying_SGD,Selling_SGD,Buying_JPY,Selling_JPY,Buying_CNY,Selling_CNY
0,2005-01-03 00:00:00,103.52,104.57,196.92,199.69,138.77,141.08,89.46,91.46,NaN,NaN,79.87,81.61,NaN,NaN,1.0026,1.0206,NaN,NaN
1,2005-01-04 00:00:00,102.64,103.94,195.22,198.53,137.77,140.46,88.87,91.24,NaN,NaN,79.2,81.22,NaN,NaN,0.9949,1.0167,NaN,NaN
2,2005-01-05 00:00:00,100.12,101.76,188.23,191.99,132.47,135.33,85.17,87.55,NaN,NaN,76.19,78.24,NaN,NaN,0.9554,0.9774,NaN,NaN
3,2005-01-06 00:00:00,98.53,100.33,185.42,189.45,130.35,133.4,83.86,86.4,NaN,NaN,74.98,77.05,NaN,NaN,0.9437,0.9678,NaN,NaN
4,2005-01-07 00:00:00,98.58,100.24,184.49,188.64,129.38,132.71,83.34,86.06,NaN,NaN,74.59,76.93,NaN,NaN,0.9358,0.96117,NaN,NaN
5,2005-01-10 00:00:00,97.59,99.54,182.29,186.85,127.12,130.65,81.83,84.66,NaN,NaN,73.43,75.91,NaN,NaN,0.9281,0.956,NaN,NaN
6,2005-01-11 00:00:00,97.4,99.11,182.61,186.75,127.36,130.58,82.17,84.81,NaN,NaN,73.52,75.82,NaN,NaN,0.9318,0.9575,NaN,NaN
7,2005-01-12 00:00:00,97.47,98.94,182.65,186.04,127.48,130.03,81.96,84.16,NaN,NaN,73.78,75.57,NaN,NaN,0.9393,0.9602,NaN,NaN
8,2005-01-13 00:00:00,97.58,99.05,184.08,187.49,128.98,131.58,83.08,85.32,NaN,NaN,74.5,76.3,NaN,NaN,0.9484,0.9698,NaN,NaN
9,2005-01-17 00:00:00,97.76,99.17,182.5,186.15,127.65,130.55,82.18,84.64,NaN,NaN,73.75,75.92,NaN,NaN,0.9533,0.9775,NaN,NaN


In [16]:
import pandas as pd

# First, ensure that the 'Date' column is treated as string
df1_cleaned['Date'] = df1_cleaned['Date'].astype(str)

# Convert to datetime with errors='coerce' to handle invalid formats
df1_cleaned['Date'] = pd.to_datetime(df1_cleaned['Date'], errors='coerce')

# Drop rows where the 'Date' column is NaT (invalid dates)
df1_cleaned = df1_cleaned.dropna(subset=['Date'])


In [17]:
df1_cleaned.to_csv('test.csv', index=False)

## Speeding up the python CSV reading processes 